<a href="https://colab.research.google.com/github/semishen/ML100Days/blob/master/Day_029_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [39]:
# 取一個類別型欄位 Sex, 與一個數值型欄位 Fare, 做群聚編碼
# Fare 對 Sex 做群聚編碼
df['Cabin'] = df['Cabin'].fillna('None')
df_mean = df.groupby('Cabin')['Fare'].mean().reset_index()
df_mode = df.groupby('Cabin')['Fare'].apply(lambda x: x.mode()[0]).reset_index()
df_median = df.groupby('Cabin')['Fare'].median().reset_index()
df_max = df.groupby('Cabin')['Fare'].max().reset_index()

temp_df = pd.merge(df_mean, df_mode, on='Cabin', how='left')
temp_df = pd.merge(temp_df, df_median, on='Cabin', how='left')
temp_df = pd.merge(temp_df, df_max, on='Cabin', how='left')

temp_df.columns = ['Cabin', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
temp_df = pd.merge(df, temp_df, on='Cabin', how='left')
temp_df.head()



,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,19.157325,8.0500,10.5000,512.3292
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,71.283300,71.2833,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,19.157325,8.0500,10.5000,512.3292
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,53.100000,53.1000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,19.157325,8.0500,10.5000,512.3292


In [0]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = (temp_df.dtypes == 'int64') | (temp_df.dtypes == 'float64')
num_features = temp_df.columns[num_features].values
num_df = temp_df.loc[:, num_features]

# 削減文字型欄位, 只剩數值型欄位
num_df = num_df.fillna(-1)
# for dtype, feature in zip(df.dtypes, df.columns):
#     if dtype == 'float64' or dtype == 'int64':
#         num_features.append(feature)
# print(f'{len(num_features)} Numeric Features : {num_features}\n')


MMEncoder = MinMaxScaler()

# 作業2
### Q: 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
### A: 沒有改善。用 Fare 對 Cabin 作 group by encoding 並沒有增加 model 的預測能力，有可能 Fare 或 Cabin 對生還率預測來說不是重要特徵。 

In [44]:
# 原始特徵 + 邏輯斯迴歸
num_raw_df = num_df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'], axis = 1)
train_X = MMEncoder.fit_transform(num_raw_df)
estimator = LogisticRegression()
logistic_reg_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('=== raw numerical features ===')
print('logistic_reg_score: ', logistic_reg_score)
num_raw_df.head()

=== raw numerical features ===
logistic_reg_score:  0.70041428661101


,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [45]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(num_df)
estimator = LogisticRegression()
logistic_reg_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('=== raw numerical features + Group by encoding ===')
print('logistic_reg_score: ', logistic_reg_score)
num_df.head()

=== raw numerical features + Group by encoding ===
logistic_reg_score:  0.694802586152784


,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,512.3292
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,512.3292
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,512.3292
